In [1]:
!pip install transformers torch

In [2]:
pip install transformers torch --upgrade

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: transformers in c:\users\suryansh\anaconda3\lib\site-packages (4.57.1)
  Using cached torch-2.9.1-cp313-cp313-win_amd64.whl.metadata (30 kB)
Using cached torch-2.9.1-cp313-cp313-win_amd64.whl (110.9 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.8.0
    Uninstalling torch-2.8.0:
      Successfully uninstalled torch-2.8.0



ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.8.0 requires torch==2.8.0, but you have torch 2.9.1 which is incompatible.
torchvision 0.23.0 requires torch==2.8.0, but you have torch 2.9.1 which is incompatible.


In [3]:
pip install tf-keras torch torchvision torchaudio transformers datasets

  Using cached torch-2.8.0-cp313-cp313-win_amd64.whl.metadata (30 kB)
Using cached torch-2.8.0-cp313-cp313-win_amd64.whl (241.3 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.9.1
    Uninstalling torch-2.9.1:
      Successfully uninstalled torch-2.9.1
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch

In [5]:
data = {
    'text': [
        "The government announced a new healthcare policy today.",
        "Scientists discovered a cure for COVID-19 instantly using lemon juice.",
        "Apple unveiled its latest iPhone with AI-powered camera.",
        "Aliens landed in New York, according to a viral post.",
        "The Prime Minister held a meeting with tech CEOs to discuss innovation."
    ],
    'label': [0, 1, 0, 1, 0]   # 0 = Real, 1 = Fake
}

df = pd.DataFrame(data)
print("✅ Dataset Loaded Successfully!\n")
print(df, "\n")

✅ Dataset Loaded Successfully!

                                                text  label
0  The government announced a new healthcare poli...      0
1  Scientists discovered a cure for COVID-19 inst...      1
2  Apple unveiled its latest iPhone with AI-power...      0
3  Aliens landed in New York, according to a vira...      1
4  The Prime Minister held a meeting with tech CE...      0 



In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

encodings = tokenizer(
    list(df['text']),
    truncation=True,
    padding=True,
    max_length=64,
    return_tensors='pt'
)

labels = torch.tensor(df['label'].values)

In [7]:
train_size = int(0.8 * len(df))
train_encodings = {key: val[:train_size] for key, val in encodings.items()}
test_encodings = {key: val[train_size:] for key, val in encodings.items()}
train_labels = labels[:train_size]
test_labels = labels[train_size:]

In [8]:
class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = NewsDataset(train_encodings, train_labels)
test_dataset = NewsDataset(test_encodings, test_labels)


In [9]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
pip install accelerate>=0.26.0

Note: you may need to restart the kernel to use updated packages.


In [11]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    logging_dir='./logs',
    logging_steps=10
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()

C:\Users\suryansh\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\suryansh\AppData\Local\Temp\ipykernel_23332\3689975084.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
C:\Users\suryansh\AppData\Local\Temp\ipykernel_23332\3689975084.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


Step,Training Loss


TrainOutput(global_step=4, training_loss=0.5633382201194763, metrics={'train_runtime': 10.2294, 'train_samples_per_second': 0.782, 'train_steps_per_second': 0.391, 'total_flos': 65777763840.0, 'train_loss': 0.5633382201194763, 'epoch': 2.0})

In [12]:
predictions = trainer.predict(test_dataset)
preds = torch.argmax(torch.tensor(predictions.predictions), dim=1)

print("\n📈 Evaluation Results:")
print("Accuracy:", round(accuracy_score(test_labels, preds), 3))
print("Classification Report:\n", classification_report(test_labels, preds))

C:\Users\suryansh\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\suryansh\AppData\Local\Temp\ipykernel_23332\3689975084.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
C:\Users\suryansh\AppData\Local\Temp\ipykernel_23332\3689975084.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])



📈 Evaluation Results:
Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         1

    accuracy                           1.00         1
   macro avg       1.00      1.00      1.00         1
weighted avg       1.00      1.00      1.00         1



In [13]:
def predict_news(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=64)
    outputs = model(**inputs)
    pred = torch.argmax(outputs.logits, dim=1).item()
    return "🟢 REAL News" if pred == 0 else "🔴 FAKE News"

print("\n💬 Demo Predictions:")
demo_texts = [
    "NASA announces water found on Mars surface.",
    "Chocolate cures all diseases, scientists confirm."
]
for t in demo_texts:
    print(f"Text: {t}\nPrediction: {predict_news(t)}\n")


💬 Demo Predictions:
Text: NASA announces water found on Mars surface.
Prediction: 🟢 REAL News

Text: Chocolate cures all diseases, scientists confirm.
Prediction: 🔴 FAKE News

